In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataset-of-songs-in-spotify/genres_v2.csv
/kaggle/input/dataset-of-songs-in-spotify/playlists.csv


In [2]:
#Make data frames from data
genres = pd.read_csv('../input/dataset-of-songs-in-spotify/genres_v2.csv')
playlists=pd.read_csv('../input/dataset-of-songs-in-spotify/playlists.csv')

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
##This function was pulled from offline to let me see where any NA values to figure out how I'd deal with them
def assess_NA(data):
    """
    Returns a pandas dataframe denoting the total number of NA values and the percentage of NA values in each column.
    The column names are noted on the index.
    
    Parameters
    ----------
    data: dataframe
    """
    # pandas series denoting features and the sum of their null values
    null_sum = data.isnull().sum()# instantiate columns for missing data
    total = null_sum.sort_values(ascending=False)
    percent = ( ((null_sum / len(data.index))*100).round(2) ).sort_values(ascending=False)
    
    # concatenate along the columns to create the complete dataframe
    df_NA = pd.concat([total, percent], axis=1, keys=['Number of NA', 'Percent NA'])
    
    # drop rows that don't have any missing data; omit if you want to keep all rows
    df_NA = df_NA[ (df_NA.T != 0).any() ]
    
    return df_NA

In [4]:
assess_NA(genres)
#Looking at where NAs are

,Number of NA,Percent NA
title,21525,50.88
Unnamed: 0,21525,50.88
song_name,20786,49.13


Omit 3 columns with 50% NAN values. Separate Predictors from class (genre)

In [5]:
om = ['title','Unnamed: 0','song_name','genre'] #Columns to omit from predictors, based on having too many NAs to be useful

X = genres.drop(om, axis = 1) #create dataframe of predictors
y = genres['genre'].to_frame() #Creating target class data frame

In [6]:
from sklearn import preprocessing

In [7]:
gen_trans = preprocessing.OrdinalEncoder().fit(y) #Encoder to turn classes to integers

In [8]:
trans_y = pd.DataFrame(gen_trans.transform(y)) #Turn classes to integers

In [9]:
X.columns #Letting myself see what columns are in predictors

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms',
       'time_signature'],
      dtype='object')

Checking for diversity of columns. If the values in each row are all the same, or all different then it's not good for predictions. will omit homogeous or overly unique columns.

In [10]:
div_check = {} #Checking for diversity of columns. If the values in each row are all the same, or all different then it's not good for predictions. will omit homogeous or overly unique columns.
for col in X:
    l = len(set(X[col]))
    div_check[col] =(l, l/42305)
    

In [11]:
div_check #Looking at diversity

{'danceability': (921, 0.021770476303037467),
 'energy': (955, 0.0225741638104243),
 'key': (12, 0.0002836544143718237),
 'loudness': (12105, 0.2861363904975771),
 'mode': (2, 4.727573572863728e-05),
 'speechiness': (1474, 0.03484221723200567),
 'acousticness': (4707, 0.11126344403734784),
 'instrumentalness': (4867, 0.11504550289563881),
 'liveness': (1737, 0.041058976480321474),
 'valence': (1716, 0.04056258125517078),
 'tempo': (15569, 0.3680179647795769),
 'type': (1, 2.363786786431864e-05),
 'id': (35877, 0.8480557853681598),
 'uri': (35877, 0.8480557853681598),
 'track_href': (35877, 0.8480557853681598),
 'analysis_url': (35877, 0.8480557853681598),
 'duration_ms': (26261, 0.6207540479848718),
 'time_signature': (4, 9.455147145727456e-05)}

In [12]:
#Finding columns to omit, and which columns are categorical
om = []
cat=[] 

for key in div_check:
    if div_check[key][1] > .5 or div_check[key][0] == 1:
        
        om.append(key)
    elif div_check[key][0]<50:
        cat.append(key)

In [13]:
#Omitting columns marked for omission
X_clean = X.drop(om, axis = 1)

In [14]:
#Separating categorical and continuous predictors for different preprocessing
categ = X_clean[cat]
contin = X_clean.drop(cat,axis=1)

In [15]:
#Fitting scaler to continuous predictors
scaler = preprocessing.StandardScaler().fit(contin)

In [16]:
#Scaling continuous predictors
contin_scaled = pd.DataFrame(scaler.transform(contin),columns=contin.columns)

In [17]:
#Fitting one hot encoder to categorical predictors
oh_enc = preprocessing.OneHotEncoder().fit(categ)


In [18]:
#Getting names of categorical columns, to put back into encoded dataframe
oh_names = oh_enc.get_feature_names()

In [19]:
#One hot encoding categorical predictors
cat_encoded = pd.DataFrame(oh_enc.transform(categ).toarray(),columns = oh_names)

In [20]:
#Fitting scaler to one hot encoded categorical predictors
cat_scale = preprocessing.StandardScaler().fit(cat_encoded)

In [21]:
#Scaling encoded categorical predictors
cat_enc_scale = pd.DataFrame(cat_scale.transform(cat_encoded),columns = oh_names)

In [22]:
#Putting categorical and continuous predictors back together, now that they're processed
a=[cat_enc_scale,contin_scaled]
X_clean_tran = pd.concat(a,axis=1)

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
#Split data into test and training
X_train, X_test, y_train, y_test = train_test_split(X_clean_tran,trans_y, test_size= .2, random_state = 345)

In [25]:
#Just looking at training data
X_train

,x0_0,x0_1,x0_2,x0_3,x0_4,x0_5,x0_6,x0_7,x0_8,x0_9,...,x2_5,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
15113,-0.298919,-0.465596,-0.278594,-0.147433,-0.243502,3.623524,-0.310226,-0.335278,-0.293014,-0.288664,...,-0.110355,0.476554,-0.982019,-0.274234,-0.441960,-0.487403,-0.763370,-0.778473,-0.081909,-2.292160
219,-0.298919,-0.465596,3.589450,-0.147433,-0.243502,-0.275974,-0.310226,-0.335278,-0.293014,-0.288664,...,9.061675,-2.307338,-1.591307,-2.922875,-0.720957,-0.562416,-0.757032,-0.376362,-1.198988,-0.398965
27450,-0.298919,2.147784,-0.278594,-0.147433,-0.243502,-0.275974,-0.310226,-0.335278,-0.293014,-0.288664,...,-0.110355,0.463784,1.264733,0.047411,-0.621089,-0.384374,1.469719,-0.663991,-1.302763,-0.648458
28916,-0.298919,-0.465596,-0.278594,-0.147433,4.106737,-0.275974,-0.310226,-0.335278,-0.293014,-0.288664,...,-0.110355,-0.749380,0.557523,0.959305,-0.708275,-0.201730,-0.763370,1.525974,-0.883804,-0.730784
4770,-0.298919,-0.465596,3.589450,-0.147433,-0.243502,-0.275974,-0.310226,-0.335278,-0.293014,-0.288664,...,-0.110355,-0.672759,-0.976578,-0.750241,-0.854114,-0.409546,-0.763370,1.178541,-0.909534,-1.739493
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32515,-0.298919,-0.465596,-0.278594,-0.147433,4.106737,-0.275974,-0.310226,-0.335278,-0.293014,-0.288664,...,-0.110355,-0.634448,1.096091,0.414617,-0.816069,-0.562841,1.458932,-0.017538,-0.836634,-0.521761
6980,-0.298919,-0.465596,-0.278594,-0.147433,-0.243502,-0.275974,-0.310226,-0.335278,3.412805,-0.288664,...,-0.110355,0.929894,0.628244,0.477178,0.360155,0.438107,-0.763370,-0.063103,0.539882,-0.983967
29771,-0.298919,-0.465596,-0.278594,-0.147433,-0.243502,-0.275974,-0.310226,-0.335278,-0.293014,-0.288664,...,-0.110355,-0.468437,0.742486,-0.745481,-0.584629,-0.548167,1.038205,0.603284,-0.699411,-0.564287
18639,-0.298919,-0.465596,3.589450,-0.147433,-0.243502,-0.275974,-0.310226,-0.335278,-0.293014,-0.288664,...,-0.110355,-0.608908,-0.742655,-1.046726,1.564912,1.257662,-0.763370,-0.615578,0.179672,0.449916


In [26]:
from sklearn.linear_model import LogisticRegression

In [27]:
#Fit a logistic regression to the training data
clf = LogisticRegression(random_state=987).fit(X_train,y_train.to_numpy().ravel())

In [28]:
#Predict test data based on logistic fit
y_hat_clf = pd.DataFrame(clf.predict(X_test))

In [29]:
#Reset indices for easy concatenation
y_hat_clf.reset_index(drop=True,inplace=True)
y_test.reset_index(drop=True,inplace=True)

In [31]:
#Reversing encoding of test labels and prediction, in case I want to use them later
y_act = pd.DataFrame(gen_trans.inverse_transform(y_test))
y_clf = pd.DataFrame(gen_trans.inverse_transform(y_hat_clf))

In [33]:
#Concatenate logistic prediction and actual labels, naming them appropriately
results_df = pd.concat([y_hat_clf,y_test],axis=1)
results_df.columns = ['clf', 'actual']

In [35]:
#Use crosstab to make a confusion matrix myself
conf_mat = pd.crosstab(results_df['clf'],results_df['actual'],rownames=['Predicted'],colnames=['Actual'])

In [36]:
#Looking at confusion matrix
conf_mat

Actual,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0
Predicted,,,,,,,,,,,,,,,
0.0,305,15,58,2,19,33,66,108,36,22,40,7,26,36,27
1.0,23,150,23,14,6,28,19,19,9,59,0,1,0,10,15
2.0,22,9,118,8,10,49,15,100,3,1,0,0,0,0,0
3.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4.0,9,6,30,4,99,26,9,63,0,0,0,0,0,2,0
5.0,31,34,57,17,16,99,10,44,3,0,0,2,0,1,1
6.0,7,1,5,0,4,3,29,20,2,5,0,0,0,5,6
7.0,237,13,257,17,176,135,147,732,7,2,3,30,4,5,33
8.0,58,52,17,7,0,19,27,32,438,14,2,0,0,0,13


In [53]:
import plotly.express as px

In [64]:
#A heat map to help visualize results. Bright spots should appear along diagonals, with dark spots everywhere else. This would indicate Perfect categorization. 
#Issue with this is that places with higher raw number of values show up brighter , even if they may have lower percentage of accuracy. I'll scale it to account for this.
px.imshow(conf_mat)

In [73]:
#Doing previously mentioned scaling
scaled_conf_mat = pd.DataFrame(preprocessing.StandardScaler().fit_transform(conf_mat))

In [74]:
#Scaled heat map. Bright spots indicate the most frequent classified thing in each column
#It's now clear that the most easily classified are class 8 and up. Class 1 is also fairly easily classified, but a lot of things between 0 and 6 are falsely classified as class 7. 
#7 is mostly classified truly positively as well.
px.imshow(scaled_conf_mat)

In [47]:
#A list of the class labels corresponding to their encoded numbers
pd.DataFrame(gen_trans.inverse_transform(pd.DataFrame(range(15))))

,0
0,Dark Trap
1,Emo
2,Hiphop
3,Pop
4,Rap
5,RnB
6,Trap Metal
7,Underground Rap
8,dnb
9,hardstyle


In [37]:
#Calculate percentages that were predicted accurately for every class
correct_class = list(np.diag(conf_mat))
actual_sums = list(conf_mat.sum(axis=0))
corr_per = pd.DataFrame(np.divide(correct_class,actual_sums)).transpose()

[305, 150, 118, 0, 99, 99, 29, 732, 438, 404, 453, 443, 463, 414, 402]

In [40]:
#View confusion matrix with fraction correct tacked onto the bottom
pd.concat([conf_mat,corr_per],axis=0)

,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0
0.0,305.000000,15.000000,58.000000,2.0,19.000000,33.000000,66.000000,108.00000,36.000000,22.000000,40.000000,7.000000,26.000000,36.000000,27.000000
1.0,23.000000,150.000000,23.000000,14.0,6.000000,28.000000,19.000000,19.00000,9.000000,59.000000,0.000000,1.000000,0.000000,10.000000,15.000000
2.0,22.000000,9.000000,118.000000,8.0,10.000000,49.000000,15.000000,100.00000,3.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
4.0,9.000000,6.000000,30.000000,4.0,99.000000,26.000000,9.000000,63.00000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000
5.0,31.000000,34.000000,57.000000,17.0,16.000000,99.000000,10.000000,44.00000,3.000000,0.000000,0.000000,2.000000,0.000000,1.000000,1.000000
6.0,7.000000,1.000000,5.000000,0.0,4.000000,3.000000,29.000000,20.00000,2.000000,5.000000,0.000000,0.000000,0.000000,5.000000,6.000000
7.0,237.000000,13.000000,257.000000,17.0,176.000000,135.000000,147.000000,732.00000,7.000000,2.000000,3.000000,30.000000,4.000000,5.000000,33.000000
8.0,58.000000,52.000000,17.000000,7.0,0.000000,19.000000,27.000000,32.00000,438.000000,14.000000,2.000000,0.000000,0.000000,0.000000,13.000000
9.0,39.000000,37.000000,2.000000,1.0,0.000000,3.000000,18.000000,7.00000,40.000000,404.000000,5.000000,2.000000,0.000000,34.000000,81.000000


Categories 6 through 14 are most easily classifiable. Corresponding to:
6	Trap Metal
7	Underground Rap
8	dnb
9	hardstyle
10	psytrance
11	techhouse
12	techno
13	trance
14	trap


In [41]:
from sklearn.metrics import confusion_matrix

In [42]:
#Just seeing that I can also do confusion matrix with sklearn metrics, but I already named the axes on the other one
pd.DataFrame(confusion_matrix(y_clf,y_act))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,305,15,58,2,19,33,66,108,36,22,40,7,26,36,27
1,23,150,23,14,6,28,19,19,9,59,0,1,0,10,15
2,22,9,118,8,10,49,15,100,3,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,9,6,30,4,99,26,9,63,0,0,0,0,0,2,0
5,31,34,57,17,16,99,10,44,3,0,0,2,0,1,1
6,7,1,5,0,4,3,29,20,2,5,0,0,0,5,6
7,237,13,257,17,176,135,147,732,7,2,3,30,4,5,33
8,58,52,17,7,0,19,27,32,438,14,2,0,0,0,13
9,39,37,2,1,0,3,18,7,40,404,5,2,0,34,81


In [43]:
from sklearn.metrics import roc_auc_score 
from sklearn.metrics import balanced_accuracy_score

In [44]:
#Calculated for use in ROC score
y_score = clf.predict_proba(X_test)

In [45]:
#Balanced accuracy calculated for comparison
balanced_accuracy_score(y_test,y_hat_clf)

0.48197107467168554

In [46]:
#ROC_AUC score calculated. This is a great score, well classified
roc_auc_score(y_test,y_score,multi_class = 'ovr')

0.9068306514411864

In [48]:
from sklearn.decomposition import PCA

In [49]:
#PCA analysis is attempted to see if it'll be possible to visually display the categories on a scatterplot
pca = PCA().fit(X_clean_tran)

In [50]:
#Seeing how much variance is explained by each principal component
pca_rat = list(pca.explained_variance_ratio_)

In [51]:
import itertools

In [52]:
#Getting a cumiulative version of the previous variance explainability
cum_rat = list(itertools.accumulate(pca_rat))

In [54]:
#Visualizing that the first couple of principal components will not be enough to scatterplot easily distinguishable classes. Not enough variability will be captured in them. First 3 principal components
#only account for 22% of variability. Things will still be jumbled together. It would require 15 values to show comfortably and there's no easy way to show a 15 dimensional graph
px.bar(cum_rat)

This shows that it requires 15 of the PCA components to explain 80% of the variance, so I won't be able to clearly visualize the distinction by plotting a few PCA values

In [55]:
from sklearn.neighbors import KNeighborsClassifier

In [56]:
#knn_fits =[]
#knn_scores = []


#for k in range(501,601,10):
#    knn = KNeighborsClassifier(k).fit(X_train,np.ravel(y_train))
#    knn_fits.append(knn)
#    knn_scores.append(knn.score(X_test,np.ravel(y_test)))
    

In [57]:
#knn = KNeighborsClassifier(1).fit(X_train,y_train)


In [58]:
#knn.predict(X_test)


In [59]:
#knn_scores